In [0]:
%tensorflow_version 2.x
%matplotlib inline
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import (Input, Dense,  Flatten, BatchNormalization,  Conv2D,  Dropout, MaxPooling2D, UpSampling2D)
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.layers import LeakyReLU, concatenate
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg  
import random
import pandas as pd

from google.colab import drive, files 
import os
import numpy as np




In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Параметры

Работа с данными

In [4]:
print(os.listdir('gdrive/My Drive/cityscapes'))
#df = pd.read_csv('um_000000.txt', sep=' ')
#print(df.transpose())

['images', 'labels', 'save2dir', 'second_try', 'third_try', 'forth_try', '5']


Определение функций

In [0]:
def create_cnn():
    img_shape = (256,512,3)
    filters = (64,64,128,256,512,256,128,64,64)
    #filters =(0, 1, 2,  3,  4, ,5  ,6  ,7 ,8)

    
    inputs = Input(shape=img_shape)
    chanDim = -1
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
    
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = LeakyReLU(alpha=0.1)(x)
        x = BatchNormalization(axis=chanDim)(x)
        if i in (1,2,3):
            x = MaxPooling2D(pool_size=(2, 2))(x)
        elif i in (5,6,7):
            x = UpSampling2D(size=(2, 2))(x)
        if i in (2,3,4,5):
            x = Dropout(0.2)(x)
        
    x = Conv2D(1, 1, padding="same",activation = 'softmax')(x)
     
    model = Model(inputs, x)
    
    opt = Adam(lr=1e-3)
    model.compile(optimizer=opt, loss='mean_absolute_error',  metrics=['accuracy','mse'])
    return model
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return K.mean( (2. * intersection + smooth) / (union + smooth), axis=0)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def unet(pretrained_weights = None,input_size = (256,512,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss=dice_coef_loss, metrics=[dice_coef,'accuracy'])
    
    #model.summary()


    return model


TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 2
N_SAMPLES = 2975 

def kappa(a):
    #return a[:,:,0]
    return a[:,:,0]

def data_generators():
    global TRAIN_BATCH_SIZE, VAL_BATCH_SIZE
    train_datagen = ImageDataGenerator( rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./33) 
    #, preprocessing_function = kappa
    target = (256,512)

    seed = 1

    x_train_generator = train_datagen.flow_from_directory(
        r'gdrive/My Drive/cityscapes/images/train',
        target_size=target,
        batch_size=TRAIN_BATCH_SIZE,
        class_mode=None,
        seed=seed)
    
    y_train_generator = test_datagen.flow_from_directory(
        r'gdrive/My Drive/cityscapes/labels/train_labels',
        target_size=target,
        batch_size=TRAIN_BATCH_SIZE,
        class_mode=None,
        seed=seed,
        color_mode = 'grayscale')
    
    x_val_generator = train_datagen.flow_from_directory(
        r'gdrive/My Drive/cityscapes/images/val',
        target_size=target,
        batch_size=VAL_BATCH_SIZE,
        class_mode=None,
        seed=seed)
    
    y_val_generator = test_datagen.flow_from_directory(
        r'gdrive/My Drive/cityscapes/labels/val_labels',
        target_size=target,
        batch_size=VAL_BATCH_SIZE,
        class_mode=None,
        seed=seed,
        color_mode = 'grayscale')
       
    return x_train_generator,y_train_generator, x_val_generator, y_val_generator

class show_images(Callback):
    def __init__(self):
        self.i=1
        self.losses = []
        self.val_losses = []
        self.val_mae = []
        self.mae = []
        self.path = r'gdrive/My Drive/cityscapes/5'
        self.logs=[]
    def on_epoch_end(self, epoch, logs={}):
        
        global train_gen, x_train, y_train

        self.logs.append(logs)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_mae.append(logs.get('val_mae'))
        self.mae.append(logs.get('mae'))

        img = x_train[0][0]
        img = np.expand_dims(img, axis=0)

        y = y_train[0][0]
        y = y * 255
        y = y.astype(np.uint8)
        X = model.predict(
            img,
            use_multiprocessing=True,
            )
        A = X[0] * 255
        A = A.astype(np.uint8)

        A = A.reshape((256,512))
        
        #fig = plt.figure()


        model.save(f'{self.path}/Model_CNN.h5')
        model.save_weights(f'{self.path}/Model_weights.h5')

        fig, subplt=plt.subplots(1, 3, figsize=(12, 12))
        
        subplt[0].plot(self.losses)
        subplt[0].set(xlabel='epochs', ylabel='value', title='model loss')
        print(y[0].shape)
        subplt[1].imshow(y.reshape((256,512)), cmap='gray')
        subplt[1].set(xlabel='real', ylabel='real', title='real')

        subplt[2].imshow(A, cmap='gray')
        subplt[2].set(xlabel='predicted', ylabel='predicted', title='predicted')
        
        plt.savefig( f'{self.path}/plots{self.i}_{A.max()}.png')
        plt.show()

        self.i +=1

def train(model, train_generator, validation_generator, iterations=10, epochs=10):
    show_image = show_images()
    for i in range(iterations):
#        draw_plots = drawing_plots() 
#        rand_images_valid , rand_images_train = self.reshuffle_samples()
    
        history = model.fit_generator(
            train_generator,
            epochs=epochs,
            validation_data = validation_generator,
            steps_per_epoch = (N_SAMPLES//TRAIN_BATCH_SIZE)-1,
            validation_steps = (500//TRAIN_BATCH_SIZE)-1,
            #steps_per_epoch = 10,
            #validation_steps = 2,
            use_multiprocessing=True,
            callbacks=[show_image])
        model.save(f'CNN_{i}.h5')
        model.save_weights(f'CNN_weights_{i}.h5')




In [0]:
model = unet()
model.summary()
x_train, y_train, x_val, y_val = data_generators()
train_gen = (pair for pair in zip(x_train, y_train))
val_gen = (pair for pair in zip(x_val, y_val))
train(model,train_gen, val_gen,5,20)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 512, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 256, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [0]:
train(model,train_gen, val_gen,4,20)

In [0]:
'''
path = r'gdrive/My Drive/cityscapes/labels/test'

path_to_color = r'gdrive/My Drive/cityscapes/labels/test_color'
path_to_instance = r'gdrive/My Drive/cityscapes/labels/test_instance'
path_to_labels = r'gdrive/My Drive/cityscapes/labels/test_labels'


for subdir, dirs, files in os.walk(path):
    if i != []:
        for i in files:
            if i[-9:] == 'color.png':
                print(i)
                path_to_i =  os.path.join(subdir,i )
                path_to_new_i = os.path.join(path_to_color, i)
                os.rename( path_to_i ,path_to_new_i)
            elif i[-15:] == 'instanceIds.png':
                print(i)
                path_to_i =  os.path.join(subdir,i )
                path_to_new_i = os.path.join(path_to_instance, i)
                os.rename( path_to_i ,path_to_new_i)
            elif i[-12:] == 'labelIds.png':
                print(i)
                path_to_i =  os.path.join(subdir,i )
                path_to_new_i = os.path.join(path_to_labels, i)
                os.rename( path_to_i ,path_to_new_i)

path = r'gdrive/My Drive/cityscapes/labels/train_labels'

path_to_color = r'gdrive/My Drive/cityscapes/labels/train_labels/train_labels'

k = 0
for subdir, dirs, files in os.walk(path):
    print(files)
    if files !=[]:
        for i in files:
            print(i)
            path_to_i =  os.path.join(subdir,i )
            path_to_new_i = os.path.join(path_to_color, i)
            os.rename( path_to_i ,path_to_new_i)
        break
        

path = r'gdrive/My Drive/cityscapes/images/train/train'
l = os.listdir(path)    

for i in l:
    a = i.find('(1)')
    if a != -1:
        print(i)
        #p = os.path.join(path, i)
        #os.remove(p)    
'''

x_train[0][0].shape

In [0]:
import cv2
from google.colab.patches import cv2_imshow
path = r'gdrive/My Drive/cityscapes/images/val/val/frankfurt_000000_000294_leftImg8bit.png'
path_to_labels = r'gdrive/My Drive/cityscapes/labels/val_labels/val_labels/frankfurt_000000_000294_gtFine_labelIds.png'
label = cv2.imread(path_to_labels)
img = cv2.imread(path)
cv2_imshow(img)
img = np.expand_dims(img, axis=0)
img = img.astype(np.float32)
pred = model.predict(img)
pred = pred.astype(np.uint8)


cv2_imshow(pred[0]*(255//pred[0].max()))
cv2_imshow(label*(255//label.max()))

